In [1]:
import time
from pathlib import Path
from typing import Any
import dataclasses
import shutil

#pyvale imports
import pyvale.dataset as dataset
import pyvale.sensorsim as sens
from pyvale.mooseherder import (MooseConfig,
                                MooseRunner,
                                ExodusReader)


In [2]:
mesh_name = "mesh_square.msh"
mesh_path = "../meshes/"
output_path = "../output/interactive/interactive_moose.vtu"
input_file_path = "../input_moose/interactive/square.i"
input_path = "../input_moose/interactive/"

In [3]:
mesh_file_path = mesh_path + mesh_name
mesh_file_path_moose = input_path + mesh_name
shutil.copyfile(mesh_file_path, mesh_file_path_moose)

'../input_moose/interactive/mesh_square.msh'

In [4]:
config = {'main_path': Path.home()/ 'moose',
          'app_path': Path.home() / 'proteus',
          'app_name': 'proteus-opt'}
moose_config = MooseConfig(config)

In [5]:
moose_runner = MooseRunner(moose_config)

moose_runner.set_run_opts(n_tasks = 1,
                          n_threads = 1,
                          redirect_out = False)

In [6]:
# moose_input = Path("input/case17.i")

moose_input = Path(input_file_path) # It actually works now :)

moose_runner.set_input_file(moose_input)

In [7]:
print(moose_runner.get_arg_list())
print()

['proteus-opt', '--n-threads=1', '-i', 'square.i']



In [8]:
start_time = time.perf_counter()
moose_runner.run()
run_time = time.perf_counter() - start_time

print()
print("-"*80)
print(f'MOOSE run time = {run_time:.3f} seconds')
print("-"*80)
print()

Framework Information:
MOOSE Version:           git commit aea74f0345 on 2025-11-13
LibMesh Version:         910736a3a20aded528c3ba99371ce65980dc8e2c
PETSc Version:           3.24.0
SLEPc Version:           3.24.0
Current Time:            Fri Nov 21 10:28:35 2025
Executable Timestamp:    Mon Nov 17 09:18:38 2025

Input File(s):
  /home/wiera/Documents/compare-fem/input_moose/interactive/square.i

Command Line Argument(s):
  --n-threads=1

Checkpoint:
  Wall Time Interval:      Every 3600 s
  User Checkpoint:         Disabled
  # Checkpoints Kept:      2
  Execute On:              TIMESTEP_END 

Parallelism:
  Num Processors:          1
  Num Threads:             1

Mesh: 
  Parallel Type:           replicated
  Mesh Dimension:          2
  Spatial Dimension:       2
  Nodes:                   2017
  Elems:                   3872
  Num Subdomains:          1

Nonlinear System:
  Num DOFs:                2017
  Num Local DOFs:          2017
  Variables:               "temperature" 
  Fin

In [9]:
input_path = Path(input_path)

output_exodus = input_path / (moose_input.stem + "_out.e")
exodus_reader = ExodusReader(output_exodus)

print("\nReading exodus file with ExodusReader:")
print(output_exodus.resolve())
print()


Reading exodus file with ExodusReader:
/home/wiera/Documents/compare-fem/input_moose/interactive/square_out.e



In [10]:
# all_sim_data = exodus_reader.read_all_sim_data()
# print("SimData from 'read_all':")
# sens.SimTools.print_sim_data(all_sim_data)

# all_sim_data.node_vars['temperature']

In [11]:
read_config = exodus_reader.get_read_config()
sens.SimTools.print_dataclass_fields(read_config)

Data class fields for: <class 'pyvale.mooseherder.simdata.SimReadConfig'>
    time: <class 'bool'>
    coords: <class 'bool'>
    connect: <class 'bool'>
    sidesets: numpy.ndarray | None
    node_vars: numpy.ndarray | None
    elem_vars: list[tuple[str, int]] | None
    glob_vars: numpy.ndarray | None
    time_inds: numpy.ndarray | None



In [12]:
read_config.time = False
read_config.coords = True
read_config.connect = False
sim_data = exodus_reader.read_sim_data(read_config)

sens.SimTools.print_sim_data(sim_data)

sol_unordered = sim_data.node_vars['temperature'][:, 1]
points_unordered = sim_data.coords



sim_data.coords.shape=(2017, 3)

sim_data.connect
    None

sim_data.node_vars
keys=dict_keys(['temperature'])
    temperature.shape=(2017, 2)

sim_data.elem_vars
    None

sim_data.glob_vars
    None



In [13]:
import numpy as np
from scipy.spatial import KDTree
import pyvista as pv

res = pv.read(mesh_file_path)
points = res.points

tree2 = KDTree(points_unordered)

_, indices = tree2.query(points)

sol = sol_unordered[indices]

In [14]:
res["sol"] = sol

res.save(output_path)